In [1]:
import numpy as np 
import matplotlib.pyplot as plt

import cv2 as cv

import os
from pathlib import Path

In [2]:
ortho_control_paths = ['/home/steffenr/uxo/raw_data/original_orthos/flight_15_transparent_mosaic_group1.tif',
                       '/home/steffenr/uxo/raw_data/original_orthos/flight_14_transparent_mosaic_group1.tif']

In [3]:
def start_points(size, split_size, overlap=0):
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            if split_size == size:
                break
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points

In [4]:
def split_images(img_path, split_w, split_h, overlap=0., save_dir=None, make_empty_annotations=False):
    if save_dir is None:
        save_dir = '/'.join(img_path.split('/')[:-1]+['split'])
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    if make_empty_annotations and not os.path.exists(os.path.join(save_dir, 'images')):
        os.makedirs(os.path.join(save_dir, 'images'))
    
    if make_empty_annotations and not os.path.exists(os.path.join(save_dir, 'labels')):
        os.makedirs(os.path.join(save_dir, 'labels'))            
    
    img = cv.imread(img_path)
    img_h, img_w, _ = img.shape
    
    X_pts = start_points(img_w, split_w, overlap)
    Y_pts = start_points(img_h, split_h, overlap)
    
    count = 0
    name = img_path.split('/')[-1][:-4]+'_split'
    format = 'jpg'
    
    for i in Y_pts:
        for j in X_pts:
            split = img[i:i+split_h, j:j+split_w]
            if not make_empty_annotations:
                cv.imwrite(f'{save_dir}/{name}_{count}.{format}', split)
            else:
                cv.imwrite(f'{save_dir}/images/{name}_{count}.{format}', split)
                Path(f'{save_dir}/labels/{name}_{count}.txt').touch()
            count += 1
    

In [5]:
size = 640
overlap = 0.2

for img_path in ortho_control_paths:
    split_images(img_path, size, size, overlap, make_empty_annotations=True)

In [8]:
# organize annotations correspond with images

split_dir = '/home/steffenr/uxo/raw_data/original_orthos/split'
img_list = os.listdir(split_dir+'/images')
label_list = os.listdir(split_dir+'/labels')

delete_list = [label for label in label_list if label[:-4]+'.jpg' not in img_list]

for label in delete_list:
    os.remove(os.path.join(split_dir, 'labels', label))